<a href="https://www.kaggle.com/code/gojocloud/gojo-s-first-analysis?scriptVersionId=263912653" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


此处添加一步特征工程，包含IsAlone和FamilySize。先前的整个流程是没有特征工程的，加入后结果更加准确（9.23kaggleTitanic规划）

In [6]:
# ===========================
# 特征工程 Feature Engineering
# ===========================

# 1. 家庭大小 FamilySize = SibSp + Parch + 1 (自己也算进去)
train_data["FamilySize"] = train_data["SibSp"] + train_data["Parch"] + 1
test_data["FamilySize"]  = test_data["SibSp"] + test_data["Parch"] + 1

# 2. 是否独自一人 IsAlone (1=独自一人, 0=有人陪伴)
train_data["IsAlone"] = (train_data["FamilySize"] == 1).astype(int)
test_data["IsAlone"]  = (test_data["FamilySize"] == 1).astype(int)

# --- 可选: 打印检查一下 ---
print("FamilySize (train):\n", train_data["FamilySize"].value_counts().sort_index().head())
print("IsAlone (train):\n", train_data["IsAlone"].value_counts())
print(train_data[["SibSp", "Parch", "FamilySize", "IsAlone"]].head())


FamilySize (train):
 FamilySize
1    537
2    161
3    102
4     29
5     15
Name: count, dtype: int64
IsAlone (train):
 IsAlone
1    537
0    354
Name: count, dtype: int64
   SibSp  Parch  FamilySize  IsAlone
0      1      0           2        0
1      1      0           2        0
2      0      0           1        1
3      1      0           2        0
4      0      0           1        1


In [7]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "FamilySize", "IsAlone"]

# one-hot 编码
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# 保证列对齐
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# 建模（仅此处参数已按你之前要求优化）
model = RandomForestClassifier(
    n_estimators=150,      # OOB 拐点
    max_depth=6,           # GridSearch 最佳
    min_samples_split=12,  # GridSearch 最佳
    min_samples_leaf=7,    # GridSearch 最佳
    random_state=1
)
model.fit(X, y)
predictions = model.predict(X_test)

# 输出结果（一字不改）
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
